# Merging all data sources

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns

## 1 Target data set and population
The country, region, year, and month of this data set provides the index for all data sets.
Target variables are only recorded every three or four months, so we outer join all data to keep features for months preceding with a hunger score.  

In [2]:
scores = gpd.read_file('region_scores.geojson')
print(scores.shape)
scores.sample(3)

(4312, 6)


,ADMIN0,ADMIN1,month,year,CS,geometry
688,Sudan,North Darfur,4.0,2012.0,2.0,"POLYGON ((27.49919 18.71566, 27.49260 16.52430..."
2644,Kenya,Nakuru,1.0,2015.0,1.5,"POLYGON ((36.23953 0.20455, 36.24593 0.19481, ..."
1153,South Sudan,Jonglei,10.0,2011.0,2.0,"POLYGON ((30.50584 9.51738, 30.75601 9.49297, ..."


## 2 Weather data
Weather data are the only data set that must merged by a spatial join. To pdo this we must:  
* Join the weather data to our regions data.
* Aggregate the weather data to the region level
* Shape the index schema to match the main data set.
* Merge on the country-region-year-month index.  

After this step we may discard geometric data.

In [3]:
weather = gpd.read_file('weather/weather_gdf.geojson')

print(weather.shape)
weather.sample(3)

(7632, 13)


,month_year,country_x,station_id,tmin_mm,tmax_mm,tavg_mm,prcp_mm,country_y,name,lat,long,elevation,geometry
951,2016-04-01T00:00:00,ET,ET000063471,197.388889,319.809524,254.555556,89.26087,ET,DIREDAWA,9.60,41.850,1260.0,POINT (41.85000 9.60000)
208,2014-05-01T00:00:00,ET,ET000063331,149.545455,NaN,224.500000,81.50000,ET,GONDAR,12.55,37.417,1967.0,POINT (37.41700 12.55000)
4925,2011-06-01T00:00:00,SU,SUM00062660,288.333333,450.000000,366.200000,NaN,SU,KARIMA,18.55,31.850,249.0,POINT (31.85000 18.55000)


In [4]:
regions = gpd.read_file('regions.geojson')

print(regions.shape)
regions.head(2)

(138, 3)


,ADMIN0,ADMIN1,geometry
0,Ethiopia,Addis Ababa,"POLYGON ((38.78052 9.09681, 38.79296 9.08120, ..."
1,Ethiopia,Afar,"POLYGON ((41.79105 10.99112, 41.77117 10.99454..."


In [5]:
weather = gpd.sjoin(weather, regions, how='left', op='within')

print(weather.shape)
weather.sample(3)

(7632, 16)


,month_year,country_x,station_id,tmin_mm,tmax_mm,tavg_mm,prcp_mm,country_y,name,lat,long,elevation,geometry,index_right,ADMIN0,ADMIN1
4721,2018-06-01T00:00:00,SU,SU000062880,NaN,NaN,NaN,NaN,SU,WAU,7.700,28.017,438.0,POINT (28.01700 7.70000),37.0,South Sudan,Western Bahr el Ghazal
3435,2019-04-01T00:00:00,SU,SU000062730,195.000000,400.000000,330.000000,NaN,SU,KASSALA,15.467,36.400,500.0,POINT (36.40000 15.46700),16.0,Sudan,Kassala
1878,2009-07-01T00:00:00,KE,KE000063820,210.304348,283.619048,245.064516,17.583333,KE,MOMBASA,-4.033,39.617,55.0,POINT (39.61700 -4.03300),69.0,Kenya,Mombasa


In [6]:
dropping_cols = ['country_x', 'country_y', 'station_id', 'index_right',
                 'name', 'lat', 'long', 'geometry', 'elevation']

weather = weather.drop(columns=dropping_cols, errors='ignore')

weather.head(1)

,month_year,tmin_mm,tmax_mm,tavg_mm,prcp_mm,ADMIN0,ADMIN1
0,2009-01-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
weather_agg = weather.groupby(by=['ADMIN0', 'ADMIN1', 'month_year']).mean()

print(weather_agg.shape)
weather_agg.sample(3)

(5616, 4)


,,,tmin_mm,tmax_mm,tavg_mm,prcp_mm
ADMIN0,ADMIN1,month_year,,,,
Kenya,Mombasa,2013-06-01T00:00:00,207.291667,285.95,249.666667,31.058824
Tanzania,Tabora,2015-08-01T00:00:00,161.920000,313.00,240.516129,0.000000
Sudan,South Darfur,2016-09-01T00:00:00,NaN,NaN,272.928571,NaN


In [8]:
# check the number of non-empty rows 
weather_agg.dropna(how='all').shape

(3786, 4)

In [9]:
weather_agg.reset_index(inplace=True)

In [10]:
# convert datetime columns to mathch schema of target dataset
weather_agg['year'] = weather_agg['month_year'].map(lambda x: int(x[:4]))
weather_agg['month'] = weather_agg['month_year'].map(lambda x: int(x[5:7]))
weather_agg.drop(columns=['month_year'], inplace=True)

weather_agg.head(3)

,ADMIN0,ADMIN1,tmin_mm,tmax_mm,tavg_mm,prcp_mm,year,month
0,Ethiopia,Addis Ababa,90.818182,237.666667,160.666667,42.25,2009,1
1,Ethiopia,Addis Ababa,99.619048,252.117647,178.807692,NaN,2009,2
2,Ethiopia,Addis Ababa,112.000000,277.600000,191.421053,NaN,2009,3


In [11]:
our_index = ['ADMIN0', 'ADMIN1', 'year', 'month']

In [12]:
main = scores.merge(weather_agg, how='outer',
                  on=our_index)

print(main.shape)
main.sample(3)

(8717, 10)


,ADMIN0,ADMIN1,month,year,CS,geometry,tmin_mm,tmax_mm,tavg_mm,prcp_mm
8565,Tanzania,Tabora,5.0,2020.0,NaN,None,171.333333,291.181818,223.384615,NaN
5892,South Sudan,Upper Nile,5.0,2019.0,NaN,None,NaN,NaN,NaN,NaN
8384,Tanzania,Ruvuma,8.0,2016.0,NaN,None,122.500000,260.428571,196.035714,NaN


## 3 Conflict data
Conflict data ranges from yars 2000-2020 so must be subset to 2009-2019.

Indicies already contain a similar schema and the same region names' spelling as the target data. We only need to rename geography columns.
We perform a simple merge on the same indicies.

In [13]:
conflict = pd.read_csv('conflict/conflict.csv')

print(conflict.shape)
conflict.sample(3)

(12238, 15)


,Unnamed: 0,country,admin1,year,month,battle_cnt,protest_cnt,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal
10230,10230,Kenya,Mandera,2007,3,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
10535,10535,Kenya,Nyamira,2007,11,0,0,0,0,1,0.0,0.0,0.0,0.0,3.0
9997,9997,Kenya,Vihiga,2004,6,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0


In [14]:
conflict.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
naming = {
    'country': 'ADMIN0',
    'admin1': 'ADMIN1'
}
conflict.rename(columns=naming, inplace=True)

In [16]:
conflict = conflict[conflict.year>=2009]

In [17]:
main = main.merge(conflict, how='outer',
                  on=our_index)

print(main.shape)
main.sample(3)

(12895, 20)


,ADMIN0,ADMIN1,month,year,CS,geometry,tmin_mm,tmax_mm,tavg_mm,prcp_mm,battle_cnt,protest_cnt,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal
9108,Somalia,Gedo,8.0,2011.0,NaN,None,NaN,NaN,NaN,NaN,10.0,1.0,0.0,3.0,0.0,24.0,0.0,0.0,1.0,0.0
815,Sudan,River Nile,7.0,2014.0,1.5,"POLYGON ((35.13589 17.59663, 35.17620 17.55844...",NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,Kenya,Elgeyo-Marakwet,10.0,2012.0,1.5,"POLYGON ((35.69886 1.27917, 35.69833 1.27657, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# drop the geometry attribute from the dataset
main.drop(columns=['geometry'], inplace=True)

## 4 Food price data
We are using only the retail prices and dropping wholesale data.
Index schema and region names are already matched with our target data set.

In [19]:
food = pd.read_csv('food_prices/price_data_retail_only.csv')

print(food.shape)
food.sample(3)

(4781, 6)


,Unnamed: 0,ADMIN0,ADMIN1,year,month,Retail
207,402,Ethiopia,Gambela,2012,2,0.676167
846,1249,Kenya,North Eastern,2013,12,0.328254
956,1479,Kenya,Rift Valley,2012,12,0.929037


In [20]:
food.drop(columns=['Unnamed: 0'], inplace=True)

In [21]:
main = main.merge(food, how='outer', on=our_index)

print(main.shape)
main.sample(10)

(14525, 20)


,ADMIN0,ADMIN1,month,year,CS,tmin_mm,tmax_mm,tavg_mm,prcp_mm,battle_cnt,protest_cnt,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal,Retail
9554,Somalia,Lower Shabelle,8.0,2018.0,NaN,NaN,NaN,NaN,NaN,38.0,0.0,0.0,18.0,4.0,82.0,0.0,0.0,37.0,6.0,-1.305854
13318,Kenya,North Eastern,5.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.075214
10987,Kenya,Kiambu,1.0,2020.0,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
10791,South Sudan,Eastern Equatoria,4.0,2020.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,NaN
9610,Somalia,Lower Shabelle,2.0,2012.0,NaN,NaN,NaN,NaN,NaN,17.0,1.0,0.0,5.0,0.0,10.0,0.0,0.0,7.0,0.0,-0.344589
970,Sudan,West Darfur,7.0,2014.0,1.5,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,2.082553
6055,Sudan,Blue Nile,8.0,2013.0,NaN,221.416667,311.9375,253.809524,470.0,3.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.850251
2850,Kenya,Nyeri,4.0,2013.0,1.5,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7264,Sudan,South Darfur,6.0,2014.0,NaN,NaN,405.0000,289.000000,NaN,1.0,3.0,0.0,0.0,17.0,6.0,0.0,0.0,0.0,3.0,3.002085
13075,Kenya,Coast,7.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575704


## 5 Population data

In [22]:
population = pd.read_csv('population/population.csv')
print(population.shape)
population.sample(3)

(4312, 7)


,Unnamed: 0,country,admin1,month,year,CS,pop
2733,2733,Kenya,Narok,1.0,2014,1.5,850920.0
2950,2950,Kenya,Taita Taveta,7.0,2009,2.5,284657.0
620,620,Sudan,Kassala,1.0,2013,1.5,2174000.0


Remove extraneous columns.

In [23]:
population.drop(columns=['Unnamed: 0'], inplace=True)
population.head(1)

,country,admin1,month,year,CS,pop
0,Ethiopia,Addis Ababa,10.0,2018,1.0,3804000.0


Rename column names to match.

In [24]:
population.rename(columns=naming, inplace=True)
population.drop(columns=['CS', 'month'], inplace=True)
population.drop_duplicates(inplace=True)

In [25]:
main = main.merge(population, 
                    on=['ADMIN0', 'ADMIN1', 'year'],
                    how='left')

print(main.shape)
main.sample(3)

(14525, 21)


,ADMIN0,ADMIN1,month,year,CS,tmin_mm,tmax_mm,tavg_mm,prcp_mm,battle_cnt,...,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal,Retail,pop
12205,Kenya,Vihiga,8.0,2015.0,NaN,NaN,NaN,NaN,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,554622.0
9113,Somalia,Gedo,12.0,2010.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
1439,Uganda,Central,4.0,2013.0,1.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


## 6 Clean up
Check for row duplicates and empty rows (those with NaN for all non-index columns).

In [26]:
# how many unique unit rows do we have
main.drop_duplicates().shape

(14525, 21)

Drop empty rows.

In [27]:
variables = ['CS', 'pop', 'tmin_mm',
       'tmax_mm', 'tavg_mm', 'prcp_mm', 'battle_cnt', 'protest_cnt',
       'riot_cnt', 'explosion_cnt', 'violence_on_civs_cnt',
       'battle_fatal', 'protest_fatal', 'riot_fatal', 'explosion_fatal',
       'violence_on_civs_fatal', 'Retail']
main.dropna(how='all', subset=variables, inplace=True)
main.shape

(14188, 21)

In [28]:
# how many unique unit rows do we have now
main.drop_duplicates().shape

(14188, 21)

In [29]:
main[(main.ADMIN1=='Oromia') & (main.year==2016)]

,ADMIN0,ADMIN1,month,year,CS,tmin_mm,tmax_mm,tavg_mm,prcp_mm,battle_cnt,...,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal,Retail,pop
224,Ethiopia,Oromia,2.0,2016.0,2.285714,147.520833,300.166667,233.692857,36.250000,7.0,...,15.0,1.0,18.0,28.0,84.0,1.0,0.0,41.0,0.233935,37489000.0
240,Ethiopia,Oromia,6.0,2016.0,2.000000,139.840000,267.500000,191.002137,70.050435,7.0,...,4.0,0.0,6.0,54.0,1.0,0.0,0.0,9.0,0.501668,37489000.0
251,Ethiopia,Oromia,10.0,2016.0,1.800000,135.694960,278.333333,195.708565,58.257895,2.0,...,25.0,3.0,15.0,0.0,5.0,76.0,0.0,336.0,0.249649,37489000.0
4690,Ethiopia,Oromia,1.0,2016.0,NaN,128.791209,278.846154,206.318750,15.000000,12.0,...,2.0,1.0,10.0,65.0,45.0,0.0,0.0,6.0,0.366545,37489000.0
4691,Ethiopia,Oromia,3.0,2016.0,NaN,142.285714,316.076923,232.140756,56.261905,8.0,...,3.0,3.0,9.0,45.0,21.0,1.0,10.0,3.0,0.183446,37489000.0
4692,Ethiopia,Oromia,4.0,2016.0,NaN,155.077922,287.130435,214.629808,91.537879,1.0,...,0.0,0.0,2.0,12.0,1.0,0.0,0.0,1.0,0.370502,37489000.0
4693,Ethiopia,Oromia,5.0,2016.0,NaN,146.707407,281.464286,192.570490,111.009972,2.0,...,4.0,1.0,4.0,8.0,2.0,7.0,0.0,1.0,0.556610,37489000.0
4694,Ethiopia,Oromia,7.0,2016.0,NaN,144.266667,250.866667,180.922697,75.638889,4.0,...,19.0,1.0,11.0,27.0,5.0,9.0,0.0,14.0,0.296089,37489000.0
4695,Ethiopia,Oromia,8.0,2016.0,NaN,139.888636,257.652174,182.386957,109.316770,5.0,...,7.0,1.0,8.0,20.0,188.0,26.0,5.0,13.0,0.291290,37489000.0
4696,Ethiopia,Oromia,9.0,2016.0,NaN,135.979348,263.333333,187.622143,103.501812,5.0,...,5.0,1.0,12.0,23.0,2.0,0.0,0.0,16.0,0.330499,37489000.0


It appears we no longer have any duplicate or empty rows.

## Write to disc

In [30]:
main.to_csv('total_data.csv')